In [1]:
# import fitz  # PyMuPDF
from PyPDF2 import PdfReader
import os
from nltk.tokenize import sent_tokenize

# def extract_text_from_pdf(pdf_path):
#     """Extracts text from a PDF file."""
#     doc = fitz.open(pdf_path)
#     text = ""
#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         text += page.get_text()
#     return text

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file with pypdf2."""
    reader = PdfReader(pdf_path)
    text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text
# Example usage:
pdf_folder = 'data/'
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# create a dictionary to store the text of each pdf and the metadata and if there is a txt with the same name
contex = {}
for pdf_file in pdf_files:
    contex[pdf_file]={}
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    contex[pdf_file]['pdf'] = text
    # get the metadata
    #metadata = fitz.open(pdf_path).metadata
    metadata = PdfReader(pdf_path).metadata
    contex[pdf_file]['metadata'] = metadata
    # check if there is a txt with the same name
    txt_path = os.path.join(pdf_folder, pdf_file.replace('.pdf', '.txt'))
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            contex[pdf_file]['transcript'] = file.read()

    else:
        contex[pdf_file]['transcript'] = None

In [2]:
# one file:
pdf_folder = 'data/batch/'
metting_name = "gib_mcp_rgc_min__2024-04-09__01"
# read the info from a xlsx file
import pandas as pd
df = pd.read_excel("data/meetingmap.xlsx")
df = df.set_index('standard name')
# get the info of the meeting
meeting_info = df.loc[metting_name+".pdf"]
# return all the columns for the meeting
meeting_info = meeting_info.to_dict()
meeting_info


{'location': 'gibsons',
 'location type': 'municipality',
 'meeting type': 'regular_council',
 'data type': 'minutes',
 'meeting date': Timestamp('2024-04-09 00:00:00'),
 'transcript': 'Yes',
 'comment': nan}

In [3]:
import fitz  # PyMuPDF
import os
from nltk.tokenize import sent_tokenize

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Example usage:
pdf_files = [metting_name+".pdf"]

# create a dictionary to store the text of each pdf and the metadata and if there is a txt with the same name
contex = {}
for pdf_file in pdf_files:
    contex[pdf_file]={}
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    contex[pdf_file]['pdf'] = text
    # get the metadata
    metadata = fitz.open(pdf_path).metadata
    contex[pdf_file]['metadata'] = metadata
    # check if there is a txt with the same name
    txt_path = os.path.join(pdf_folder, pdf_file.replace('.pdf', '.txt'))
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            contex[pdf_file]['transcript'] = file.read()

    else:
        contex[pdf_file]['transcript'] = None

In [4]:
contex[pdf_files[0]].keys()

dict_keys(['pdf', 'metadata', 'transcript'])

In [43]:
def chunk_text(text, chunk_size=512, overlap=256):
    """Divides text into overlapping chunks."""
    sentences = sent_tokenize(text)
    chunks = []
    chunk = []

    current_length = 0
    for sentence in sentences:
        chunk.append(sentence)
        current_length += len(sentence.split())

        if current_length >= chunk_size:
            chunks.append(" ".join(chunk))
            chunk = chunk[-(overlap // len(sentence.split())):]  # Start next chunk with the overlap
            current_length = len(" ".join(chunk).split())

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks

# Example usage:
input_text = [contex[pdf_file]['pdf'] for pdf_file in pdf_files]
for text in input_text:
    chunks = chunk_text(text)

In [8]:
# better option
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=50
)
input_text = [contex[pdf_file]['pdf'] for pdf_file in pdf_files]
for text in input_text:
    all_splits = text_splitter.split_text(text)


In [9]:
len(all_splits)


2

In [ ]:
# from langchain_community.embeddings import GPT4AllEmbeddings
# from langchain_community.vectorstores import Chroma
# model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
# gpt4all_kwargs = {'allow_download': 'True'}

# embedding = GPT4AllEmbeddings(  model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs)
# # Index
# vectorstore = Chroma.from_documents(
#     documents=all_splits,
#     collection_name="rag-chroma",
#     embedding=embedding,
# )
# retriever = vectorstore.as_retriever()

In [10]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
import json

In [96]:
# model_name = "llama3"
model_name = "phi3"


model = Ollama(model=model_name)

PROMPT_llama ="""<|system|>
You are an assitance that find distinct proposals titles in a meeting note. You are provided with a context delimited by ### as a part of a meeting note. The goal is to find unique individual proposals. 
DO NOT use `\ n` to indicate a new proposals. Only seperate them by ||. Only write the number of proposal once after your thinking step followed by ||. 
Your task is to find the count of unique proposal and write the title of the proposal(s) delimited by ||. If a proposal is not complete (does not have the result), do not count it. DO NOT write less than 5 or more than 15 words for each proposal.  

Your output style should be this: 
<thinking steps> || <num proposal, n: int > || <proposal 1 title: str> || <proposal 2 title: str> ... || <proposal n title: str>

Let's think step by step. Here are the steps to solve the task:
1. Find the pattern in the chunk that indicates a proposal.
2. Find the number of unique proposals in the chunk. 
3. Make sure all content about each proposal including the title, proposer, seconder, and the result are included and they are unique.
4. Write a title (min 5 to max 15 words) for those proposals.

Here is an example: 
Context: ### Minutes of the Council meeting of February 6, 2024, be approved.\n CARRIED UNANIMOUSLY \n Council Meeting\n Minutes, February 27, 2024 3\n 3. Council (City Finance and Services) \n MOVED by Councillor Dominato\n SECONDED by Councillor Carr\n THAT the Minutes of the Council meeting following the Standing Committee on City\n Finance and Services meeting of February 7, 2024, be approved.\n CARRIED UNANIMOUSLY\n 4. Court of Revision (Business Improvement Areas) - February 8, 2024\n MOVED by Councillor Bligh\n SECONDED by Councillor Zhou\n  THAT the Minutes of the Court of Revision (Business Improvement Areas) meeting of\n February 8, 2024, be approved.\n CARRIED UNANIMOUSLY\n MATTERS ADOPTED ON CONSENT\n MOVED by Councillor Carr \n Tax Rates Bylaw”:\n THAT “Tax Rates Bylaw, 2024, No. 9017” be considered.\n R2024-04-22/10\n BYLAW FIRST,   ###

Thinking steps:
1. It is constantly using the word THAT to indicate a proposal. 
2. Found 4 proposals which has THAT 
3. The last proposal does not have the result, so it is not a complete proposal. 
4. I have 3 proposals, and will write short title for them. 

assitante response:
|| 3 ||  the Minutes of the Council meeting of February 6, 2024, be approved. || the Minutes of the Council meeting following the Standing Committee on City Finance and Services, be approved || the Minutes of the Court of Revision (Business Improvement Areas) meeting be approved.
<|end|>

<|user|>
Context:
###
{context}
###
<|end|>

<|assistant|> """ 

PROMPT_phi ="""<|system|>
You are an assitance that find all proposals mentioned in a meeting note. You are provided with a context delimited by ### as a part of a meeting note.
Your task is to find the exact count of proposal, and then write short title of each proposal(s) delimited by <|sep|>. 

Let's think step by step. Here are the steps to solve the task:
1. Find the pattern in the chunk that indicates a proposal.
2. Find the count of all the proposals in the chunk: n. 
3. Make sure all content about each proposal including the title, proposer, seconder, and the result (carried or rejected) are included in the context.
4. Write a short title (min 5 to max 15 words) for n proposals.

RULES:
- Your answer must not include any speculation or inference. Do not assume or change content. Only provide information that is explicitly stated in the context.
- The number of used delimiter <|sep|> MUST BE EQUAL to the number of proposals found.
- Each proposal title MUST be between 5 to 15 words. If it was longer, summarize it.
- Only write the number of proposal once in the begining. 
- NEVER use `\n` in your response.
- An answer is considered grounded if **all** information in **every** sentence in the answer is **explicitly** mentioned in the source context, **no** extra information is added and **no** inferred information is added.

Sample of correct assistant output:
3 <|sep|>  title 1 ... <|sep|>  title 2 .... <|sep|> title 3 ...

<|end|>

<|user|>
Context:
###
{context}
###
<|end|>

<|assistant|> """ 
if model_name == "llama3":
    print("llama")
    PROMPT = PROMPT_llama
    delimiter = "||"
elif model_name == "phi3":
    print("phi")
    PROMPT = PROMPT_phi
    delimiter = "<|sep|>"


prompt = ChatPromptTemplate.from_template(PROMPT)

chain = prompt | model
proposal_dictionary = {}
proposal_count_dictionary = {}
total_chunks = len(all_splits)
iterator = 1
while iterator <= total_chunks:
    context = all_splits[iterator-1]
    print(f'Processing chunk {iterator} of {total_chunks} ...')
    latest_proposal = chain.invoke({'context': context})
    # parse the latest_proposal and seperate them if it has special token <sep>
    latest_proposal = latest_proposal.split(delimiter) 
    if len(latest_proposal) == 0:
        print(f"There was an error in chunk {iterator}, no {delimiter} found, running again")
        continue
    try:
        s = 0
        int(latest_proposal[0])
    except:
        try:
            s = 1
            int(latest_proposal[1])
            
        except:
            print(f"There was an error in chunk {iterator}, running again")
            print(latest_proposal[1])
            continue
    print(f"{latest_proposal[s]} new proposals was found in this chunk")
    proposal_dictionary[iterator-1] = latest_proposal[s+1:]
    # checking for their lenght 
    for i in reversed(range(len(proposal_dictionary[iterator-1]))):
        if len(proposal_dictionary[iterator-1][i]) < 20:
            # remove this proposal
            print(f" Removing proposal (lenght limit) '{proposal_dictionary[iterator-1][i]}'")
            proposal_dictionary[iterator-1].pop(i)
    proposal_count_dictionary[iterator-1] = int(latest_proposal[s])
    if len(proposal_dictionary[iterator-1]) != proposal_count_dictionary[iterator-1]:
        print(f"Warning, proposals in chunk {iterator} does not match the count")
    iterator += 1


phi
Processing chunk 1 of 2 ...
3  new proposals was found in this chunk
Warning, proposals in chunk 1 does not match the count
Processing chunk 2 of 2 ...
3  new proposals was found in this chunk
Warning, proposals in chunk 2 does not match the count


In [97]:
latest_proposal

['3 ',
 ' Parcel Tax Roll Review Panel Setup  \n',
 ' Canada BC Housing Benefit MOU Execution  \n',
 ' CEPF Funding Support  \n',
 ' Community Event Fund Approval and Reinstatement']

In [98]:
# convert proposal_dictionary to all_proposals list
all_proposals = []
chunks =[]
for  key, value in proposal_dictionary.items():
    all_proposals.extend(value)
    # populate chunk with the keys on all elements of the value
    chunks.extend([key for _ in range(len(value)) ])
print(len(all_proposals))
print(chunks)

11
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [99]:
# find the count for each proposal in the chunks
# count the number of elements in the chunks
proposal_count_returned = {i:chunks.count(i) for i in set(chunks)}
# find the chunks that the two dictionaries are different
diff = {k: proposal_count_returned[k] - proposal_count_dictionary[k] for k in proposal_count_returned if k in proposal_count_dictionary and proposal_count_returned[k] != proposal_count_dictionary[k]}

diff

{0: 4, 1: 1}

In [100]:
# if the is differece, we need to look at the proposals that are different
from sklearn.feature_extraction.text import TfidfVectorizer

def get_similarity_score(proposals):
    pairwise_similarity = {}
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    # select two adjacent proposals and calculate the cosine similarity
    for i in range(len(proposals)-1):
        tfidf = vect.fit_transform([proposals[i], proposals[i+1]])                                                                                                                                                                                                                       
        pairwise_similarity[f'{i}-{i+1}']=(tfidf * tfidf.T).toarray()[0][1]
    return pairwise_similarity

In [101]:
for k in diff.keys():
    print(f"Checking the proposals in chunk {k}")
    print(get_similarity_score(proposal_dictionary[k]))


Checking the proposals in chunk 0
{'0-1': 0.2169338170113969, '1-2': 0.12083223046959846, '2-3': 0.03367858515950911, '3-4': 0.03367858515950911, '4-5': 0.2243746584076321, '5-6': 0.06655924660794856}
Checking the proposals in chunk 1
{'0-1': 0.0, '1-2': 0.0, '2-3': 0.0}


In [27]:
# save the proposal dictionary to a txt file
with open('data/proposals_{}.txt'.format(metting_name), 'w') as file:
    json.dump(proposal_dictionary, file)

In [18]:
# load the proposal dictionary from a txt file
with open('data/proposals_{}.txt'.format(metting_name), 'r') as file:
    proposal_dictionary = json.load(file)

In [102]:
# we need to check if the proposals are unique
# we can use the cosine similarity to check if the proposals are unique

# get the cosine similarity between the proposals
from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer(min_df=1, stop_words="english")
to_remove = []
# select two adjacent proposals and calculate the cosine similarity
for i in range(len(all_proposals)-1):
    tfidf = vect.fit_transform([all_proposals[i], all_proposals[i+1]])                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 
    if pairwise_similarity.toarray()[0][1] > 0.5:
        to_remove.append(i+1)
        print(f"Proposal {i} and {i+1} are similar with score {pairwise_similarity.toarray()[0][1]}")
        print(all_proposals[i])
        print(all_proposals[i+1])  
        print('-------------------')                                                                                                                                                                                             


In [103]:
# remove the similar proposals
unique_proposals = [all_proposals[i] for i in range(len(all_proposals)) if i not in to_remove]
unique_chunks = [chunks[i] for i in range(len(chunks)) if i not in to_remove]

In [104]:
# convert back to dictionary
unique_proposal_dictionary = {}
for i in range(len(unique_chunks)):
    if unique_chunks[i] in unique_proposal_dictionary:
        unique_proposal_dictionary[unique_chunks[i]].append(unique_proposals[i])
    else:
        unique_proposal_dictionary[unique_chunks[i]] = [unique_proposals[i]]

In [105]:
unique_proposal_dictionary

{0: [' Proposal to adopt Regular Business Agenda of April 9, 2024 R2024-63 ',
  ' Proposal for adoption of minutes from March 19, 2024 R2024-64 ',
  ' Amendment to Development Permit Delegation Authority Bylaw No.1054-04, 2024 R2024-65 ',
  ' Integration of 5-year capital plan into the financial plan with modifications except for Dog Park and Pickleball projects; approval of Healthy Harbours Project budget funded by surplus R2024-66 ',
  ' Annual tax rates increase by 8% (3% general operations, 5% policing costs) Bylaw No. R2024-67 ',
  ' Approval of the Financial Plan and Annual Tax Rate Bylaw for 2024-2028 R2024-68 ',
  ' Review Panel on Parcel Taxes R2024-69'],
 1: [' Parcel Tax Roll Review Panel Setup  \n',
  ' Canada BC Housing Benefit MOU Execution  \n',
  ' CEPF Funding Support  \n',
  ' Community Event Fund Approval and Reinstatement']}

In [106]:
# get all the information for this specific meeting from the proposals_for_review.xlsx file (training only) sheet review
df = pd.read_excel("data/proposals_for_review.xlsx", sheet_name="review")
df = df.set_index('standard_name')
# get the list of proposals where the meeting name is the same as the current meeting
meeting_proposals = df.loc[metting_name+".pdf"]

meeting_proposals

,sectionid,subsectionid,policy_prop,theme,terms,policy_prop_future_date,future_action_date_type,vote_result,theme_as_reviewed,terms_to_remove,terms_to_add,comments,Wrong Cut,future_date_as_reviewed,sentence_suggesting_future_action,voting_result_as_reviewed,Validated?
standard_name,,,,,,,,,,,,,,,,,
gib_mcp_rgc_min__2024-04-09__01.pdf,4.0,1.0,R2024-66 2024-2028 Preliminary General Service...,unclassified,the Dog\nPark and Pickleball projects,NaN,NaN,CARRIED,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Yes
gib_mcp_rgc_min__2024-04-09__01.pdf,5.0,1.0,R2024-69 Parcel Tax Roll Review Panel - Water/...,housing,community,2024-05-07,exact,CARRIED,drop,community,NaN,NaN,NaN,NaT,NaN,NaN,Yes
gib_mcp_rgc_min__2024-04-09__01.pdf,6.0,3.0,R2024-72 Allocation of Community Event Fund:\n...,housing,community,NaN,NaN,CARRIED,drop,community,NaN,NaN,NaN,NaT,NaN,NaN,Yes


In [107]:
df = pd.read_excel("data/proposals_for_review.xlsx", sheet_name="themes")
themes = df['theme'].tolist()
themes = ['Uncategorized', 'food', 'housing', 'children', 'climate', 'health', 'environment', 'indigenous', 'women', 'diversity', 'poverty', 'psychoactive', 'mental']


In [171]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from typing import Optional

# model_name = "phi3"
model_name = "llama3"


# Schema for structured response
class Proposal(BaseModel):
    section: str = Field(description="The section and subsection of the proposal" , required=True)
    title: str = Field(description="A short title of the proposal", required=True)
    policy_prop: str = Field(description="Full proposal content", required=True)
    theme: str = Field(description=f"categorical value from themes: {themes}", required=True)
    vote_result: str = Field(description="The result of the vote", required=True)
    future_date: Optional[str] = Field(description="The future date of the proposal")



# Prompt template
PROMPT_llama =""" <|system|> Use tool_calls from OllamaFunctions of langchain_experimental.llms.ollama_functions.OllamaFunctions
 The output should be in json format which is structured with the following schema:

    section: str = Field(description="The section and subsection of the proposal" , required=True)
    title: str = Field(description="A short title of the proposal", required=True)
    policy_prop: str = Field(description="Full proposal content", required=True)
    theme: str = Field(description=f"categorical value from themes", required=True)
    vote_result: str = Field(description="The result of the vote", required=True)
    future_date: Optional[str] = Field(description="The future date of the proposal")
  

Except the future_date, all other fields are required.
    
RULES: 
1. The given proposal  is delimted by *** and followed after the PROPOSAL word. 
2. The  context  is delimited by ### and followed after the CONTEXT word.
3. The themes is delimited by @@@ and followed after the THEMES word.
4. Call the tool_calls OllamaFunctions of langchain_experimental.llms.ollama_functions.OllamaFunctions and format json.

Example of correct assistant JSON output:

section: "5.2",
title: " Development 4 story building.",
policy_prop: "THAT to development a 4story bulding for mixed use with 5 rooms.",
theme: "Housing",
vote_result: "CARRIED UNANIMOUSLY",
future_date: "February 6, 2024"

<|end|>

<|user|>
THEMES:
@@@
{themes}
@@@

Context:
###
{context}
###

PROPOSAL:
###
{proposal}
###

Human: Given the context, retrive section, title, policy_prop, theme, vote_result and future_date of the given proposal.
<|end|>

<|assistant|> """


# Prompt template
PROMPT_phi =""" <|system|> Call AIMessage with the tool_calls from OllamaFunctions of langchain_experimental.llms.ollama_functions.OllamaFunctions with the model phi3
 the output should be in json format. Make sure you have all the required information. The model is structured with the following schema:

section: Optional[str] = Field(description="The section and subsection of the proposal" )
title: str = Field(description="A short title of the proposal", required=True)
policy_prop: str = Field(description="Full proposal content", required=True)
theme: str = Field(description=f"a single string value from THEMES given by user", required=True)
vote_result: Optional[str] = Field(description="The result of the vote")
future_date: Optional[str] = Field(description="The future date of the proposal")


Example of correct assistant JSON output:

section: "5.2",
title: " Development 4 story building.",
policy_prop: "THAT to development a 4story bulding for mixed use with 5 rooms.",
theme: "Housing",
vote_result: "CARRIED UNANIMOUSLY",
future_date: "February 6, 2024"

RULES:
- your output MUST HAVE the exact json format as the schema. DO NOT add any other keys, information.
- Your answer must not include any speculation or inference. Do not assume or change dates and times. Only provide information that is explicitly stated in the context.
- The proposal is delimtied by ***, context delimited by ###, and THEMES is after @@@.
- An answer is considered grounded if **all** information in **every** sentence in the answer is **explicitly** mentioned in the source context, **no** extra information is added and **no** inferred information is added.
- theme field is a **string** value selected from one of the THEMES. Only return a string. 
<|end|>

<|user|>
THEMES:
@@@
{themes}
@@@

Context:
###
{context}
###

PROPOSAL:
###
{proposal}
###

Human: Given the context, retrive section, title, policy_prop, theme, vote_result and future_date of the given proposal. 
<|end|>

<|assistant|> """

if model_name == "llama3":
    print("llama")
    PROMPT = PROMPT_llama
elif model_name == "phi3":
    print("phi")
    PROMPT = PROMPT_phi

prompt = ChatPromptTemplate.from_template(PROMPT)

# Chain
llm = OllamaFunctions(model=model_name,format="json", keep_alive=-1, temperature=0)
structured_llm = llm.with_structured_output(Proposal)
chain = prompt | structured_llm

iterator = 0
context = all_splits[iterator]
proposal = unique_proposal_dictionary[iterator][0]
themes_str = ', '.join(themes)

result = chain.invoke({'themes':themes_str ,'context': context, 'proposal': proposal})
result


llama


ValueError: Failed to parse a response from llama3 output: { "name": "R2024-63" }

  





 

In [120]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json", temperature=0
    )

class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

prompt = PromptTemplate.from_template(
    """  <|system|> Call AIMessage with the tool_calls from OllamaFunctions of langchain_experimental.llms.ollama_functions.OllamaFunctions with the model phi3
 the output should be in json format. The model is structured with the following schema:

    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

  <|end|> 
    <|user|>
Context:
Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde.

Human: {question}
<|end|>

<|assistant|> """
)

structured_llm = model.with_structured_output(Person)
chain = prompt | structured_llm

alex = chain.invoke("Describe Alex")
alex

Person(name='Alex', height=5.0, hair_color='blonde')